Use this notebook to generate the transmission loss patterns for various sediment layers, source depth, and source sound frequency.

**Disclaimer:** This calculation requires `uwlib` and `fimpack`, which are proprietary modules developed at BYU.

I will try to make this notebook accessible for those who don't have access to these packages, by exporting what I calculated and include them in this repo.
However, the calculation to generate the transmission loss data will also be included here.

In [ ]:
from pathlib import Path
import tarfile

import numpy as np

# import uwlib
# from fimpack import orca_manager

%matplotlib inline
plt.style.use("default")

In [ ]:
# Setting up directories
WORK_DIR = Path().absolute()
DATA_DIR = WORK_DIR / "data"
SVP_DIR = DATA_DIR / "svp"

In [ ]:
# To help those who don't have access to uwlib and fimpack, let's try to get the
# transmission loss data from the tarball that I generated.
TL_DIR = DATA_DIR / "transmission_loss"
tarball = DATA_DIR / "transmission_loss.tar.gz"
if not TL_DIR.exists():
    # Check if tarball exists. If it does, then extract.
    if tarball.exists():
        os.chdir(DATA_DIR)  # Entering the data directory
        # Extracting the tarball
        with tarfile.open(tarball.name) as f:
            f.extractall()
        os.chdir(WORK_DIR)  # Exiting datadirectory, go back to the previous path
    else:
        # There is no transmission loss data. This is mainly for me to generate the
        # data.
        TL_DIR.mkdir()

In [ ]:
# List of TOML files
opt_toml_file = WORK_DIR / "FIM_opt.toml"
# Sound profile files
sediment_type_list = ["mud", "clay", "silt", "sand", "gravel"]
svp_toml_path_dict = {
    sed: SVP_DIR / f"svp_{sed}_35m_unit_test.toml" for sed in sediment_type_list
}
# List of sound source frequencies
freq_list = [50, 100, 200, 400]  # In Hz

In [ ]:
# Source depth and range, and receiver depth
source_depth = np.array([8, 16])
source_range = np.linspace(1, 5, 21)
receiver_depth = np.arange(5, 76, 5)

source_range_fine = np.linspace(0, source_range[-1], 501)[1:]
receiver_depth_fine = np.linspace(0, receiver_depth[-1], 751)[1:]

In [ ]:
# Run the calculation to generate TL data for all sediment types and frequencies
for sediment_type in sediment_type_list:
    for freq in freq_list:
        TL_file = TL_DIR / f"TL_{sediment_type}_f{freq}Hz.npz"
        if not TL_file.exists():
            print(
                f"Compute transmission loss for {sediment_type} at frequency {freq} Hz"
            )
            # Initialize ORCA
            svp_toml_path = svp_toml_path_dict[sediment_type]
            orca = orca_manager.initialize_orca(
                freq, str(svp_toml_path), str(opt_toml_file)
            )
            # Compute transmission loss
            TL, _, _ = uwlib.tl.calc_tl_from_orca(
                orca,
                freq,
                source_range_fine,
                source_depth,
                receiver_depth_fine,
                return_gf=True,
                return_nmode=True,
            )
            # Export
            np.savez(
                TL_file,
                source_depth=source_depth,
                source_range=source_range_fine,
                receiver_depth=receiver_depth_fine,
                TL=TL,
            )

In [ ]:
# Create a tarball
if not tarball.exists():
    os.chdir(DATA_DIR)  # Entering the data directory
    # Creating a tarball
    with tarfile.open(tarball, "w:gz") as tar:
        tar.add(TL_DIR.name)
    os.chdir(WORK_DIR)  # Exiting datadirectory, go back to the previous path